In [ ]:
import socket
import threading
import tkinter as tk
from tkinter import scrolledtext, messagebox, simpledialog, filedialog
import os

class ChatServer:
    def __init__(self, host="127.0.0.1", port=55555):
        self.server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.server.bind((host, port))
        self.server.listen()

        self.clients = []
        self.nicknames = []
        self.conversations = []
        self.rules = self.load_rules()

        print(f"Server started on {host}:{port}")

        # GUI setup
        self.root = tk.Tk()
        self.root.title("Chat Server")
        self.root.configure(bg='#282C34')

        # Frame for buttons and client list
        self.right_frame = tk.Frame(self.root, bg='#282C34')
        self.right_frame.pack(side=tk.RIGHT, fill=tk.Y, padx=10, pady=10)

        # Client list area with scrollbar
        self.client_list_area = scrolledtext.ScrolledText(
            self.right_frame, wrap=tk.WORD, state='disabled',
            bg='#1E1E1E', fg='#00FF00', font=('Consolas', 10), height=15, width=30
        )
        self.client_list_area.pack(pady=5, padx=5)

        # Disconnect Button
        self.disconnect_button = tk.Button(
            self.right_frame, text="Disconnect", command=self.disconnect_client,
            bg='#E06C75', fg='white', font=('Consolas', 10, 'bold'), width=15
        )
        self.disconnect_button.pack(pady=5)

        # Client List Button
        self.client_list_button = tk.Button(
            self.right_frame, text="Refresh", command=self.refresh_client_list,
            bg='#61AFEF', fg='white', font=('Consolas', 10, 'bold'), width=15
        )
        self.client_list_button.pack(pady=5)

        # Attach File Button
        self.attach_file_button = tk.Button(
            self.right_frame, text="Send File", command=self.attach_and_send_file,
            bg='#98C379', fg='white', font=('Consolas', 10, 'bold'), width=15
        )
        self.attach_file_button.pack(pady=5)

        # Current Conversation Button
        self.current_conversation_button = tk.Button(
            self.right_frame, text="Monitor", command=self.show_conversations,
            bg='#D19A66', fg='white', font=('Consolas', 10, 'bold'), width=15
        )
        self.current_conversation_button.pack(pady=5)

        # Save Conversation Button
        self.save_conversation_button = tk.Button(
            self.right_frame, text="Save", command=self.save_conversations,
            bg='#C678DD', fg='white', font=('Consolas', 10, 'bold'), width=15
        )
        self.save_conversation_button.pack(pady=5)

        # Broadcast Message Button
        self.broadcast_message_button = tk.Button(
            self.right_frame, text="Broadcast", command=self.broadcast_from_server,
            bg='#56B6C2', fg='white', font=('Consolas', 10, 'bold'), width=15
        )
        self.broadcast_message_button.pack(pady=5)

        threading.Thread(target=self.receive_connections, daemon=True).start()

        self.root.protocol("WM_DELETE_WINDOW", self.on_close)
        self.root.mainloop()

    def load_rules(self):
        rules = """
        1. For the list of connected clients, type '@list' and send to the server.
        2. For a private conversation with any client, type '@' followed by their nickname and then the message.
        3. To send a message to the server, type '@server' and your message.
        """
        return rules

    def log_message(self, message):
        print(message)
        self.conversations.append(message)

    def broadcast(self, message, sender_client=None):
        self.log_message(message)
        for client in self.clients:
            if client != sender_client:
                try:
                    client.send(message.encode('ascii'))
                except:
                    self.remove_client(client)

    def broadcast_file(self, file_path):
        filename = os.path.basename(file_path)
        file_size = os.path.getsize(file_path)
        file_metadata = f"FILE|{filename}|{file_size}"

        with open(file_path, 'rb') as file:
            file_data = file.read()

        for client in self.clients:
            try:
                client.send(file_metadata.encode('ascii'))
                client.send(file_data)
            except:
                self.remove_client(client)

    def send_private_message(self, recipient_nickname, message, sender_nickname):
        private_message = f"(Private) {sender_nickname} -> {recipient_nickname}: {message}"
        self.log_message(private_message)

        if recipient_nickname in self.nicknames:
            recipient_index = self.nicknames.index(recipient_nickname)
            recipient_client = self.clients[recipient_index]
            try:
                recipient_client.send(private_message.encode('ascii'))
            except:
                self.remove_client(recipient_client)
        else:
            sender_index = self.nicknames.index(sender_nickname)
            sender_client = self.clients[sender_index]
            sender_client.send(f"User {recipient_nickname} not found.".encode('ascii'))

    def remove_client(self, client):
        if client in self.clients:
            index = self.clients.index(client)
            nickname = self.nicknames[index]
            self.clients.remove(client)
            self.nicknames.remove(nickname)
            self.broadcast(f"{nickname} has left the chat.")
            client.close()
            self.log_message(f"{nickname} disconnected.")

    def handle_client(self, client):
        while True:
            try:
                message = client.recv(1024).decode('ascii')
                sender_nickname = self.nicknames[self.clients.index(client)]
                if message.startswith('@'):
                    if message == '@list':
                        client.send("\n".join(self.nicknames).encode('ascii'))
                    elif message.startswith('@server'):
                        server_message = message[8:].strip()
                        self.broadcast(f"Server message from {sender_nickname}: {server_message}")
                    else:
                        target_nickname, private_message = message[1:].split(' ', 1)
                        self.send_private_message(target_nickname, private_message, sender_nickname)
                else:
                    broadcast_message = f"{sender_nickname}: {message}"
                    self.broadcast(broadcast_message, sender_client=client)
            except:
                self.remove_client(client)
                break

    def receive_connections(self):
        self.log_message("Server started... Waiting for connections...")
        while True:
            client, address = self.server.accept()
            self.log_message(f"Connected with {address}")

            client.send(f"Connected successfully. Rules for using the chat: \n{self.rules}".encode('ascii'))
            nickname = client.recv(1024).decode('ascii')
            self.nicknames.append(nickname)
            self.clients.append(client)

            self.log_message(f"{nickname} joined the chat.")
            self.refresh_client_list()

            threading.Thread(target=self.handle_client, args=(client,), daemon=True).start()

    def disconnect_client(self):
        if not self.clients:
            messagebox.showinfo("Info", "No clients connected.")
            return

        selected_nickname = simpledialog.askstring("Disconnect Client", "Enter the nickname of the client to disconnect:")

        if not selected_nickname:
            messagebox.showwarning("Warning", "No nickname entered.")
            return

        if selected_nickname in self.nicknames:
            index = self.nicknames.index(selected_nickname)
            client_to_remove = self.clients[index]
            self.remove_client(client_to_remove)
            self.refresh_client_list()
            messagebox.showinfo("Success", f"Client '{selected_nickname}' disconnected.")
        else:
            messagebox.showerror("Error", f"Nickname '{selected_nickname}' not found.")

    def refresh_client_list(self):
        self.client_list_area.configure(state='normal')
        self.client_list_area.delete(1.0, tk.END)
        self.client_list_area.insert(tk.END, "\n".join(self.nicknames))
        self.client_list_area.configure(state='disabled')

    def attach_and_send_file(self):
        file_path = filedialog.askopenfilename()
        if file_path:
            try:
                self.broadcast_file(file_path)
                self.log_message(f"File '{os.path.basename(file_path)}' sent to all clients.")
            except Exception as e:
                messagebox.showerror("Error", f"Failed to send file: {e}")

    def broadcast_from_server(self):
        message = simpledialog.askstring("Broadcast", "Enter the message to broadcast:")
        if message:
            broadcast_message = f"Server: {message}"
            self.broadcast(broadcast_message)

    def show_conversations(self):
        conversation_window = tk.Toplevel(self.root)
        conversation_window.title("Current Conversations")
        conversation_window.configure(bg='#282C34')

        conversation_area = scrolledtext.ScrolledText(
            conversation_window, wrap=tk.WORD, state='normal',
            bg='#1E1E1E', fg='#00FF00', font=('Consolas', 10), height=20, width=60
        )
        conversation_area.pack(padx=10, pady=10, fill=tk.BOTH, expand=True)

        conversation_area.insert(tk.END, "\n".join(self.conversations))
        conversation_area.configure(state='disabled')

    def save_conversations(self):
        file_path = filedialog.asksaveasfilename(defaultextension=".txt", filetypes=[("Text files", "*.txt")])
        if file_path:
            try:
                with open(file_path, 'w') as file:
                    file.write("\n".join(self.conversations))
                messagebox.showinfo("Success", f"Conversations saved to {file_path}")
            except Exception as e:
                messagebox.showerror("Error", f"Failed to save conversations: {e}")

    def on_close(self):
        self.server.close()
        self.root.destroy()

if __name__ == "__main__":
    ChatServer()
